In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

import pandas as pd
import random
import csv
import os
import math
import numpy as np
from tqdm import tqdm
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.mode.chained_assignment = None

## Run model on clean dataset

In [6]:
df = pd.read_pickle("./pickles/df_small_clean.pkl")
df.describe()

,prop_id,srch_id,position,price_quality,competitor_bool,availability_bools,visited_before,click_bool,booking_bool,promotion_flag,prop_brand_bool,random_bool
count,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000,4945.000000
mean,71092.730637,160507.023256,16.762993,46.410301,0.161780,0.099090,0.077654,0.044287,0.028918,0.234985,0.634580,0.281496
std,40120.359438,95166.467890,10.370876,29.084732,0.368286,0.298813,0.267654,0.205753,0.167593,0.424032,0.481596,0.449775
min,119.000000,65.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36956.000000,83202.000000,8.000000,30.135000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,71026.000000,150675.000000,16.000000,40.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
75%,105165.000000,249343.000000,26.000000,54.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000
max,140795.000000,330595.000000,39.000000,524.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
y = df["position"]
# X = df.drop("position", axis=1).copy()
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_test.drop("position", axis=1)
X_test['position'] = 0

In [ ]:
rfc = RandomForestClassifier(n_jobs=1)
model = rfc.fit(X_train, y_train)
predictions = model.predict(X_test)

In [ ]:
pred_groups = X_test.groupby('srch_id').agg({'position':lambda x: list(x)})

In [ ]:
X_test['predicted_rank'] = predictions
X_test['position'] = y
X_test.to_pickle("./pickles/random_forest_small_solution.pkl")

In [ ]:
# convert to numpy array
y = df['prop_id'].to_numpy()
X = df.drop("prop_id", axis=1).copy().to_numpy()

In [11]:
@jit
def rand_for(X_train, y_train, X_test):
    rfc = RandomForestClassifier(n_jobs=1)
    model = rfc.fit(X_train, y_train)
    predictions = model.predict(X_test)
    
    return predictions

    
df = pd.read_pickle("./pickles/df_small_clean.pkl")
y = df["position"]
# X = df.drop("position", axis=1).copy()
X = df.copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

X_test.drop("position", axis=1)
X_test['position'] = 0

numpy_result = rand_for(X_train.to_numpy(), y_train.to_numpy(), X_test.to_numpy())

#df = pd.DataFrame(data=numpy_result, index=df['srch_id'], columns=["prop_id", "srch_id", "position", "price_quality", "competitor_bool", "availability_bools", "visited_before", "click_bool", "booking_bool", "promotion_flag", "prop_brand_bool", "random_bool"])

X_test['predicted_rank'] = numpy_result
X_test['position'] = y
X_test.to_pickle("./pickles/random_forest_small_solution.pkl")

<ipython-input-11-e0d5cef2fb7b>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "rand_for" failed type inference due to: Untyped global name 'RandomForestClassifier': cannot determine Numba type of <class 'abc.ABCMeta'>

File "<ipython-input-11-e0d5cef2fb7b>", line 3:
def rand_for(X_train, y_train, X_test):
    rfc = RandomForestClassifier(n_jobs=1)
    ^

  @jit
/Users/lotteheek/.pyenv/versions/3.7.4/lib/python3.7/site-packages/numba/core/object_mode_passes.py:178: NumbaWarning: Function "rand_for" was compiled in object mode without forceobj=True.

File "<ipython-input-11-e0d5cef2fb7b>", line 2:
@jit
def rand_for(X_train, y_train, X_test):
^

  state.func_ir.loc))
/Users/lotteheek/.pyenv/versions/3.7.4/lib/python3.7/site-packages/numba/core/object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more